# Basic Implementation of Handwritting recognition with MNIST dataset using TensorFlow

## Code

### Importing required files

In [1]:
import input_data # For processing MNIST images 
import tensorflow as tf

### Adding command line flags for parameters and configurations

In [2]:
tf.app.flags.DEFINE_string("log_dir","softmax_logs","Directory for saving the summaries")
tf.app.flags.DEFINE_integer("max_it",1000,"Number of iterations to run")
tf.app.flags.DEFINE_integer("batch",100,"Batch size")
FLAGS = tf.app.flags.FLAGS

print "Logging summaries to:",FLAGS.log_dir

Logging summaries to: softmax_logs


### Loading MNIST dataset from ./data directory as one hot vectors

In [4]:
mnist = input_data.read_data_sets("data/", one_hot=True)

print "Training images size :",mnist.train.images.shape
print "Training labels size :",mnist.train.labels.shape
print "Testing images size :",mnist.test.images.shape
print "Testing labels size :",mnist.test.labels.shape

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Training images size : (55000, 784)
Training labels size : (55000, 10)
Testing images size : (10000, 784)
Testing labels size : (10000, 10)


### Creating name scope in the graph for the input placeholders

Placeholders are later used to process the data in batches of required size. It is a <b>Tensor</b> that may be used as a handle for feeding a value, but not evaluated directly.

In [5]:
with tf.name_scope("Input_Data") as scope:
  x = tf.placeholder(tf.float32, [None,784], name="x")
  y_ = tf.placeholder(tf.float32, [None,10], name="y")

### Creating variables for storing weights and biases

In [6]:
W = tf.Variable(tf.zeros([784,10]), name="weights")
b = tf.Variable(tf.zeros([10]), name="bias")

### Creating name scope in the graph for the applying softmax regression

<b>TensorFlow</b> provides direct implementation of the softmax function. 

In [7]:
with tf.name_scope("Wx_b") as scope:
  y = tf.nn.softmax(tf.matmul(x,W)+b)

### Creating histogram summaries for weights, biases and output labels

In [8]:
w_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
y_hist = tf.histogram_summary("y", y)

### Creating name scope in the graph for cross entropy ,i.e, performance measure

In [9]:
with tf.name_scope("Xentropy") as scope:
  cross_entropy = -tf.reduce_sum(y_*tf.log(y))
  ce_summ = tf.scalar_summary("cross entropy", cross_entropy)

### Creating name scope in the graph for training step 

The train step operation is added using standard gradient descent optimizer minimizing the cross entropy defined above.

In [10]:
with tf.name_scope("Train") as scope:
  train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

### Creating name scope in the graph for testing step

In [11]:
with tf.name_scope("Test") as scope:
  correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  accuracy_summary = tf.scalar_summary("accuracy", accuracy)

### Preliminary steps

In the following segment we merge all the summariy operations created above to ease the append of the events to log. We initialize the variables. Then we create a <b>Session</b> object which encapsulates the environment for executing the <b>Operation</b> objects.

In [12]:
# Merging all the summaries created above
merged = tf.merge_all_summaries()

# Initializing the variables
init = tf.initialize_all_variables()

# Creating the session for the graph
sess = tf.Session()

# Running the session of the graph for the variable initialization defined above
sess.run(init)

### Initializing Summary Writer

Initializing the summary writer for the graph to the directory specified in the input flags

In [13]:
writer = tf.train.SummaryWriter(FLAGS.log_dir, sess.graph_def)

### Training iterations

Running the training steps and evaluating accuracy after every 100 iterations

In [14]:
print "Running training steps with batch size",FLAGS.batch

for i in range(FLAGS.max_it):
  if i % 100 == 0:  # Record summary data, and the accuracy

    # Getting values for the placeholders
    feed = {x: mnist.test.images, y_: mnist.test.labels}

    # Running the session of the graph for calculating the Accuracy and summaries by the feeding the data fetched above
    result = sess.run([merged, accuracy], feed_dict=feed)

    summary_str = result[0]
    acc = result[1]

    # Add summary to the summary writer after every 10 steps
    writer.add_summary(summary_str, i)

    print("Accuracy at step %s: %s" % (i, acc))
  else:
    # Getting next 10 data points for training
      batch_xs, batch_ys = mnist.train.next_batch(FLAGS.batch)
      feed = {x: batch_xs, y_: batch_ys}

    # Running the session of the graph for Gradient Descent Optimization with data points fetched above
      sess.run(train_step, feed_dict=feed)

Running training steps with batch size 100
Accuracy at step 0: 0.098
Accuracy at step 100: 0.8864
Accuracy at step 200: 0.8846
Accuracy at step 300: 0.8248
Accuracy at step 400: 0.9057
Accuracy at step 500: 0.8733
Accuracy at step 600: 0.8846
Accuracy at step 700: 0.9153
Accuracy at step 800: 0.9147
Accuracy at step 900: 0.9164


### Generating weights visualization

In [15]:
weights = tf.unpack(tf.transpose(W))
for i in range(0,10):
  w = tf.reshape(weights[i],(-1,28,28,1))
  writer.add_summary(sess.run(tf.image_summary("weight_"+str(i),w,max_images=1)))

### Evaluating the accuracy on the test dataset

In [16]:
print "Final Evaluation"
print sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})

Final Evaluation
0.9082


## Data Flow Graphs

### Basic Softmax Implementation

#### Overview Graph

![Overview Graph](./images/overview_graph.png)

#### Expanded Graph

![Expanded Graph](./images/expanded_graph.png)

## Results

### Basic Softmax Implementation

Following results were obtained with batch size = 100 and number of iterations = 1000

#### Accuracy

![Expanded Graph](./images/accuracy_basic.png)

#### Cross Entropy

![Expanded Graph](./images/xentropy_basic.png)

#### Weights Visualization

##### 0:

<img src="./images/weight_0.png" width="200" height="200" />


##### 1:

<img src="./images/weight_1.png" width="200" height="200" />


##### 2:

<img src="./images/weight_2.png" width="200" height="200" />


##### 3:

<img src="./images/weight_3.png" width="200" height="200" />


##### 4:

<img src="./images/weight_4.png" width="200" height="200" />


##### 5:

<img src="./images/weight_5.png" width="200" height="200" />


##### 6:

<img src="./images/weight_6.png" width="200" height="200" />


##### 7:

<img src="./images/weight_7.png" width="200" height="200" />


##### 8:

<img src="./images/weight_8.png" width="200" height="200" />


##### 9:

<img src="./images/weight_9.png" width="200" height="200" />
